In [1]:
import sys
import os

import numpy as np
from tqdm.notebook import tqdm

# add parent dir to be able to import utils file
parent_dir = os.path.abspath('../../')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import duqling_py.functions as functions
from duqling_py.tests.duqling_r_interface import DuqlingRInterface
duq_r = DuqlingRInterface()

In [14]:
def lhs_array(n: int, d: int, seed: int | None = None) -> np.ndarray:
    rng = np.random.default_rng(seed)
    u = rng.random((n, d))
    perms = np.column_stack([rng.permutation(n) for _ in range(d)])
    return (perms + u) / n

In [32]:
NUM_SAMPLES = 100
failed_funcs = []

stochastic_piston_kwargs = dict( 
    Ta_generate = lambda:0.5, 
    P0_generate = lambda:0.5 
)

for fname in tqdm(duq_r.quack().fname):
    try:
        func_info = duq_r.quack(fname)
        input_dim = int(func_info['input_dim'][0])
        X = lhs_array(NUM_SAMPLES, input_dim)
        
        duq_py_func = getattr(functions, fname)
        kwargs = stochastic_piston_kwargs if fname == 'stochastic_piston' else dict()
        y_r = np.column_stack([duq_r.duq(x=x, f=fname, **kwargs) for x in X])
        y_p = np.column_stack([duq_py_func(x, **kwargs) for x in X])

        assert np.isclose(y_r, y_p).all()  # output array shapes should match and values should be more or less identical
    except:
        failed_funcs.append(fname)

if failed_funcs:
    print("The following functions failed the equivalence test:")
    for f in failed_funcs:
        print('\t- '+f)

  0%|          | 0/61 [00:00<?, ?it/s]

R[write to console]: Error in if (N[t] <= 0) { : missing value where TRUE/FALSE needed

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In stats::rbinom(3, c(S[t - 1], I[t - 1], R[t - 1]), x[6:8]) :
R[write to console]: 
 
R[write to console]:  NAs produced

R[write to console]: 2: 
R[write to console]: In stats::rbinom(1, R[t - 1] - deaths[3], x[9]) :
R[write to console]:  NAs produced



The following functions failed the equivalence test:
	- ocean_circ
	- dts_sirs


## Summary of issues

- `ocean_circ` is stochastic, so the failed equivalence test isn't totally surprising
- `dts_sirs` fails to terminate early when N[t] is null ('N[t] <= 0' breaks in this case)
- `ignition` uses different equation from paper
    - The paper uses $r^5 \bigl[1 + 100000 \bigl(1 + \operatorname{erf}(10 (r − 2))\bigr)\bigr]$
    but the code uses $r^5 \bigl[1+100000 \bigl(2 \operatorname{cdf}(10\sqrt{2} (r-2))\bigr)\bigr]$